In [ ]:
import pandas as pd
import numpy as np
from mlxtend.plotting import plot_decision_regions
from sklearn.svm import SVC, SVR
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from IPython.display import Image
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# SVM

## 1. Czym są SVM i zasada działania algorytmu
## 2. Co to jest Kernel ? 
## 3. Problem klasyfikacji wielu klas
## 4. Hiperparametry modelu
***************

# 1. SVM
## Suport Vector Machines 
## Maszyny wektorów wspornych (nośnych) 

Metoda uczeni maszynowego opracowana w AT&T Bell Laboratories przez Vladimira Vapnika z kolegami w latach 90.

Sprowadza się do odpowiedzi na pytanie: jak rozdzielić dwie kategorie używając prostej linii ? 

In [ ]:
data = pd.DataFrame({'Kolor': ['CZ', 'CZ', 'CZ', 'CZ', 'CZ', 'CZ', 'CZ', 'CZ', 'CZ', 'CZ', 'NI', 'NI', 'NI', 'NI', 'NI', 'NI', 'NI', 'NI', 'NI', 'NI', 'NI'],
                     'X': [2, 4, 3.5, 4.2, 1, 4.7, 2.4, 1.7, 2.3, 3.7, 5.6, 7, 9.9, 6.8, 5.5, 8.4, 7.2, 6.1, 9, 8.2, 7.9],
                     'Y': [1, 2.3, 2, 1.5, 3, 4.8, 2.3, 3.5, 4.6, .9, 10, 8.8, 6.6, 6.8, 7.9, 5.3, 6, 8.7, 9.5, 6.6, 9]})

In [ ]:
plt.figure(figsize=(5,5));
plt.scatter(data[data['Kolor'] =='CZ']['X'], data[data['Kolor'] =='CZ']['Y']);
plt.scatter(data[data['Kolor'] =='NI']['X'], data[data['Kolor'] =='NI']['Y']);

Linii rozdzielających dane jak powyżej możemy wyznaczy wiele, w SVM chodzi o to by wybrać tą optymalną.

Znajdujemy taką linię, by "margin" (odległość między linią klasyfikacji a najbliższymi jej obserwacjami ) było jak największe. 

Obserwacje leżące najbliżej linii klasyfikacji nazywane są "support vectors" 

In [ ]:
Image("img/svm_margin.png", width=500)

Gdyby usunąć ze zbioru wszystkie obserwacje poza support vetors to klasyfikator byłby taki sam. Więc dla tego algorytmu istotne są jedynie obserwacje graniczne. Poniżej przestawimy przykład pokazujący to zjawisko. 
Zatem SVM nie jest tak podatny na outleiery. 

Zatem widzimy tutaj znaczą różncę względem regresji logistycznej.
Ponadto w przypadku SVM otrzymujemy wynik po porstu jako przynależność do klasy, nie mamy tutaj interpretacji probabilistycznej wyniku  

Ponadto w przeciwieństwie do regresji logistycznej SVM jest metodą nieparametryczną tzn. nie szukamy tutaj parametrów (wag) dla zmiennych lecz na podstawie zbioru danych będziemy szukać optymalnego podziału klas. 

**********

In [ ]:
# mapowanie etykiet, tak byśmy potem mogli skorzystać z rysowania wykresów decyzyjnych plot_decision_regions
encoder = LabelEncoder()
data['Kolor'] = encoder.fit_transform(data['Kolor'])

In [ ]:
# uczenie modelu stadarodow jak innych w pakiecie sklearn
# wybieramy kernel liniowy by pokazać podział linią prostą - o krenelach będzie w dalszej części 
simple_svm = SVC(kernel='linear')
simple_svm.fit(data[['X', 'Y']], data['Kolor']);

In [ ]:
# rysowanie wykresu decyzyjności na przetrzeni 
plot_decision_regions(X = data[['X', 'Y']].to_numpy(), y = data['Kolor'].to_numpy().astype(np.int), clf=simple_svm);

In [ ]:
data2 = pd.DataFrame({'Kolor': [  'CZ',  'CZ', 'CZ', 'CZ', 'CZ', 'NI', 'NI', 'NI', 'NI', 'NI', 'NI', 'NI'],
                     'X': [ 4.2,  4.7, 2.4, 1.7,  3.7, 5.6, 7, 9.9, 6.8, 5.5, 8.4, 7.2],
                     'Y': [   1.5, 4.8, 2.3, 3.5, .9, 10, 8.8, 6.6, 6.8, 7.9, 5.3, 6]})

data2['Kolor'] = LabelEncoder().fit_transform(data2['Kolor'])
simple_svm2 = SVC(kernel='linear').fit(data2[['X', 'Y']], data2['Kolor'])
plot_decision_regions(X = data2[['X', 'Y']].to_numpy(), y = data2['Kolor'].to_numpy().astype(np.int), clf=simple_svm2);


## Jak wygląda klasyfikator w przestrzeni ? 

In [ ]:
Image("img/hiperpłaszczyzna.png", width=850)

## Miękki margines

Dotychczas przedstawiany był twardy margines, tj. żaden punkt w zbiorze treningowym nie znajdował się między liniami wyznaczonymi przez wektory wspierające.
Jest to kontrolowane przez parametr
C (o hiperparametrach w daszej części).

Punkty wewnątrz marginesu uważamy za potencjalnie błędnie zaklasyfikowane

*********

In [ ]:
Image("img/soft_margin.png", width=600)

In [ ]:
## w pakiecie dataset mamy wiele funkcji do generowania różnych struktur danych 
df = datasets.make_blobs(n_samples=200, random_state=67, centers=2,cluster_std=3)
X = pd.DataFrame(df[0], columns=['a', 'b'])
y = df[1]
sns.scatterplot(X.a, X.b, hue = y);

In [ ]:
svm = SVC(kernel='linear').fit(X, y)
plot_decision_regions(X = X.to_numpy(), y = y, clf=svm);

## SVM dla regresji

Zasadę działania SMV możemy również zasosować w problemach regresji. Działanie to jest mniej intuicyjne niż w przypadku klasyfikacji ale zasada również polega na tym, że będzimy chcieli zminimalizować błąd przez znalezienie takiej hiperpłaszczyzy dla któej margines będzie największy. W praktyce chodzi o to by otrzymana linia regresji zawierała jak najwięcej punktów w jak najbliższej odlegości od niej.
*****


In [ ]:
df = pd.read_csv("data/Advertising.csv")
df = df.drop("Unnamed: 0", axis=1)
X = df['TV']
y = df['Sales']
df.head()

In [ ]:
srv_reg = SVR(kernel='linear').fit(X.to_numpy().reshape(-1,1), y)

def reg(x):
    y = srv_reg.predict(x)
    return y

xs = np.arange(min(X),max(X),0.01)
plt.plot(xs,reg(xs.reshape(-1,1)),color="black");
plt.scatter(X,y);

# 2.Kernel

## Mapowanie do wyższych wymiarów

In [ ]:
Image("img/mapowanie3d.png", width=800)

## Jak łatwo odseparować od siebie klasy, które wydają się być nie do odseparowania?
## Przykład liniowy

In [ ]:
X = np.linspace(-5, 5, 10)
y = np.zeros(10)
labs = np.array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0])
plt.figure(figsize=(7,5))
plt.scatter(X[labs!=1], y[labs!=1])
plt.scatter(X[labs==1], y[labs==1])
plt.ylim(bottom=-1, top=1);

### Sprowadzenie do wyższego wymiaru przez podniesienie od kwadratu

In [ ]:
x_sq = X**2

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(X[labs!=1], x_sq[labs!=1])
plt.scatter(X[labs==1], x_sq[labs==1]);

## Teraz możemy łatwo rozdzielić klasy linią prostą  

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(X[labs!=1], x_sq[labs!=1])
plt.scatter(X[labs==1], x_sq[labs==1])
plt.axhline(y=1, );

## Kernel (Jądro) jest to funkcja matematyczna przekształcająca przestrzeń obserwacji. Dzięi stosowaniu różnych funkcji możemy przekształcić problemy nieliniowe w liniowo separowalne.

Maszyna wektorów nośnych klasyfikuje dane wykorzystując niejawne przekształcenie zbioru treningowego do przestrzeni cech wyższego wymiaru. W nowej przestrzeni cech dopasowywana jest optymalna hiperpłaszczyzna

In [ ]:
Image("img/kernel.png", width=600)

# Typy kerneli - przykłady na różnych strukturach danych 

In [ ]:
df = datasets.make_circles(n_samples=500,random_state=0, noise =.2, factor=.1)
X = pd.DataFrame(df[0], columns=['a', 'b'])
y = df[1]
sns.scatterplot(X.a, X.b, hue = y);

## Liniowy (linear)


In [ ]:
lin_svm = SVC(kernel='linear',gamma = 'scale')
lin_svm.fit(X,y)
plot_decision_regions(X.to_numpy(),y, clf=lin_svm);

## Wilomianowy (poly)

### Degree = 2

In [ ]:
svm_poly2 = SVC(kernel='poly', degree=2,gamma = 'scale').fit(X,y)
plot_decision_regions(X.to_numpy(),y, clf=svm_poly2);

### Degree = 3

In [ ]:
svm_poly3 = SVC(kernel='poly', degree=3,gamma = 'scale').fit(X,y)
plot_decision_regions(X.to_numpy(),y, clf=svm_poly3);

## Radial (rbf)

In [ ]:
svm_rad = SVC(kernel='rbf',gamma = 'scale').fit(X,y)
plot_decision_regions(X.to_numpy(),y, clf=svm_rad);

## Zadanie 1. 

Dla zbioru danych pniżej stwórz modele SVM z kernelem liniowym, radial i polynominal w kilku wariantach stopni wielomianu.
Narysuj wykresy oraz policz skuteczność modeli. Który był najskuteczniejszy ? 

In [ ]:
df = datasets.make_moons(n_samples=500, random_state=0, noise = .09)
X = pd.DataFrame(df[0], columns=['a', 'b'])
y = df[1]
sns.scatterplot(X.a, X.b, hue = y);

# 3. Problem klasyfikacji wielu klas

SVM jest klasyfikatorem binarnym. Możemy jednak zastosować techniki trenowania wielu klasyfikatorów i łączenia ich w celu uzyskania złożnoego modelu obsługującego wiele klas.

Są dwa główne podejścia:
- one vs one (stosowany domyślnie w sklearn)
- one vs all (aby go użyć należy posłużyć się modelem ```sklearn.multiclass.OneVsRestClassifier```)

W pierwszym podejściu budujemy klasyfikatory dla wszystkich możliwych par klas. Jest to z reguły skuteczniejsza metoda lecz złożoność obliczeniowa rośnie nam wykładniczo wraz ze wzrostem liczby klas. 
W drugim podejściu budujemy jden klasyfikator dla każdej klasy. 
*******

In [ ]:
df = datasets.make_blobs(n_samples=500, random_state=67, centers=3,cluster_std=2)
X = pd.DataFrame(df[0], columns=['a', 'b'])
y = df[1]
sns.scatterplot(X.a, X.b, hue = y);

In [ ]:
lin_ovo = SVC(kernel='linear')
lin_ovo.fit(X,y)
plot_decision_regions(X.to_numpy(),y, clf=lin_ovo);

In [ ]:
lin_ovr = OneVsRestClassifier(SVC(kernel='linear')).fit(X,y)
plot_decision_regions(X.to_numpy(),y, clf=lin_ovr);

# 4. Hiperparmetry

Hiperparamerami nazywamy takie parametry, które definiujemy jeszcze przed rozpoczęciem trenowania. Hiperparametry nie są wyliczane przez algorytm podczas uczenia tak jak np wagi w modelu regresji liniowej. Hiperpaarametry decydują zwykle o tym w jaki sposób ma działać model jakiego rodzaju obliczenia czy przekształecenia ma zastosować, ile iteracji ma wykonać itd. 

## Regularyzacja (C)

Określa jak bardzo chcemy uniknać złego sklasyfikowania obserwacji. 
Jeżeli C jest duże, wówczas algorytm dobierze węższe marginesy. Dopasowanie będzie lesze ale możemy mieć prblem z overfttingiem.


In [ ]:
df = datasets.make_blobs(n_samples=300, random_state=67, centers=2,cluster_std=2.5)
X = pd.DataFrame(df[0], columns=['a', 'b'])
y = df[1]
sns.scatterplot(X.a, X.b, hue = y);

In [ ]:
svm = SVC(kernel='rbf',gamma='auto',C=0.1)
svm.fit(X,y)
plt.title('C = 0.1')
plot_decision_regions(X.to_numpy(),y, clf=svm);

In [ ]:
svm = SVC(kernel='rbf',gamma='auto',C=100)
svm.fit(X,y)
plt.title('C = 100')
plot_decision_regions(X.to_numpy(),y, clf=svm);

## Gamma $\gamma$

Parametr dla kerneli typu radial, poly i sigmoiod. Określa ile oberwacji będzie miało wpływ na na hiperpłaszczyznę.
Wysoka wartość parametru oznacza, że tylko kilka punktów będzie miało wpły na podział, więc przestrzeń będzie można "łatwiej wygiąć" 

Możemy też, skorzystać z wartości 'auto' oraz 'scale' - wtedy w zależności od zbioru danych algorytm dobierze nam odpowiednią wartość parametru.
Dokumentacja: 
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html


In [ ]:
Image("img/gamma.png", width=700)

In [ ]:
svm = SVC(kernel='rbf',gamma=0.05)
svm.fit(X,y)
plt.title('Gamma = 0.05')
plot_decision_regions(X.to_numpy(),y, clf=svm);

In [ ]:
svm = SVC(kernel='rbf',gamma=2)
svm.fit(X,y)
plt.title('Gamma = 2')
plot_decision_regions(X.to_numpy(),y, clf=svm);

## Zastosowanie w klasyfikacji obrazów

SVM ze względu na swoją dobrą skuteczność w rozwiązywaniu problemów nieliniowych swojego czasu był wykorzystywany w problemach związanych z klasyfikacją obrazów.

In [ ]:
mnist = pd.read_csv('data/mnist_test.csv')
## bieżemy tylko pierwsze 2000 obseracji żeby szybciej się liczyło
mnist = mnist.iloc[0:2000]

Y = mnist['label']
X = mnist.drop(columns = 'label').to_numpy()
print(len(X[0]))
print(X[0])

In [ ]:
plt.matshow(X[0].reshape(28,28));

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3,random_state=50)

In [ ]:
## regresja liniowa 
lin = LogisticRegression().fit(X_train,Y_train)
pred = lin.predict(X_test)
print('Accuracy:',accuracy_score(Y_test,pred))

In [ ]:
# SVM
svm = SVC(kernel='poly',gamma='auto',degree=3).fit(X_train,Y_train)
pred = svm.predict(X_test)
print('Accuracy:',accuracy_score(Y_test,pred))

In [ ]:
confusion_matrix(Y_test,pred)

## Zadanie 2

Zbiór mnist (pierwszych 2000 obserwacji) podziel na zbiory: treningowy walidacyjny, testowy w stosunku 60%, 20% 20%. 
Następnie spróbuj znaleźć najlepszy model SVM. Przetestuj różne kernele i ich hiperparametry (C, gamma) sprawdzając skuteczność (accuracy score) na zbiorze walidacyjnym.
Wybierz najlepszy model i na końcu policz skuteczność tego modelu na zbiorze testowym 
